In [ ]:
# project 2, Group 7

In [ ]:
# task 1

In [3]:
class OptimizationProblem:
    def __init__(self, f, grad=None):
        self.f = f
        self.grad = grad

    def evaluate(self, x):
        return float(self.f(x))

    def gradient(self, x):
        if self.grad is None:
            raise NotImplementedError("No gradient provided.")
        return np.array(self.grad(x), dtype=float)

In [ ]:
# task 2: general opitimization class

In [24]:
class OptimizationMethod:
    def __init__(self, problem, tol=1e-6, max_iter=100):
        self.problem = problem
        self.tol = tol
        self.max_iter = max_iter
        self.history = []

    def compute_direction(self, x):
        # need implemention
        raise NotImplementedError

    def line_search(self, x, s, g):
        alpha = 1.0  # will be changed in the subsequet tasks 
        return alpha

    def solve(self, x0):
        x = np.array(x0, dtype=float)
        for _ in range(self.max_iter):
            g = self.problem.gradient(x)
            if np.linalg.norm(g) < self.tol:
                break
            s = self.compute_direction(x, g)
            alpha = self.line_search(x, s, g)
            x = x + alpha * s
        return x


使用newton method来测试[task1，task2]，可以参考

In [21]:
import numpy as np
from chebyquad_problem import chebyquad, gradchebyquad

#newton method
def fd_hessian(f, x, eps=1e-6):
    n = x.size
    H = np.zeros((n, n))
    fx = f(x)
    for i in range(n):
        ei = np.zeros(n); ei[i] = 1.0
        for j in range(i, n):
            ej = np.zeros(n); ej[j] = 1.0
            if i == j:
                H[i, i] = (f(x+eps*ei) - 2*fx + f(x-eps*ei)) / (eps**2)
            else:
                fpp = f(x + eps*ei + eps*ej)
                fpm = f(x + eps*ei - eps*ej)
                fmp = f(x - eps*ei + eps*ej)
                fmm = f(x - eps*ei - eps*ej)
                H[i, j] = H[j, i] = (fpp - fpm - fmp + fmm) / (4*eps**2)
    # 对称化（Task3 要求 G := 0.5*(G + G^T)）
    return 0.5*(H + H.T)

class NewtonMethod(OptimizationMethod):
    def compute_direction(self, x, g):
        H = fd_hessian(self.problem.evaluate, x)
        # 解 H s = -g
        return -np.linalg.solve(H, g)

    # 可选：以后你可以在 Task4 替换为 exact line search
    # def line_search(self, x, s, g): ...
for n in (4, 8, 11):
    x0 = np.linspace(0, 1, n)  # 题目里常用的初始点
    prob = OptimizationProblem(chebyquad, gradchebyquad)
    x_star = NewtonMethod(prob, tol=1e-8, max_iter=200).solve(x0)
    print(f"n={n}, x*=", np.round(x_star, 6), " f(x*)=", prob.evaluate(x_star))

n=4, x*= [0.112702 0.3709   0.6291   0.887298]  f(x*)= 8.49335105474771e-32
n=8, x*= [0.051472 0.199429 0.308723 0.427707 0.572293 0.691277 0.800571 0.948528]  f(x*)= 1.2303417374350283e-18
n=11, x*= [-0.010965  0.042985  0.042985  0.042985  0.042985  0.042985  0.042985
  0.042985  0.042985  0.042985 -0.010965]  f(x*)= 2.7605204466878144
